<a href="https://colab.research.google.com/github/mikkelgolf/testrepo/blob/main/golfbox_hole_by_hole_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Remove any existing Chrome & Chromedriver
!apt-get purge google-chrome-stable
!apt-get remove -y chromium-browser chromium-chromedriver

# Install Chrome (Stable version)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y -f

# Install Chromedriver matching the installed Chrome version
!pip install chromedriver-autoinstaller
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

# Update environment variables so Colab finds Chrome & Chromedriver
import os
os.environ["PATH"] += ":/usr/bin/google-chrome:/usr/bin/chromedriver:/usr/bin/"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'chromium-browser' is not installed, so not removed
Package 'chromium-chromedriver' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
--2025-03-08 20:54:09--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.152.190, 142.250.152.91, 142.250.152.93, ...
Connecting to dl.google.com (dl.google.com)|142.250.152.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114757600 (109M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 109.44M   237MB/s    in 0.5s    

2025-03-08 20:54:10 (237 MB/s) - ‘google-chrome-stable_cu

In [7]:
# Install required packages
!pip install selenium webdriver-manager

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from google.colab import files

# Step 1: Replace with the Actual Tournament URL
TOURNAMENT_URL = "https://golfbox.no/livescoring/tour/?language=2057#/competition/4331868/holebyhole/4034534/2"

# Step 2: Set up Selenium WebDriver with options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
chrome_options.binary_location = "/usr/bin/google-chrome-stable"

# Set correct Chromedriver path
chromedriver_path = "/usr/local/lib/python3.11/dist-packages/chromedriver_autoinstaller/134/chromedriver"

# Initialize WebDriver
try:
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=chrome_options)
    driver.set_page_load_timeout(15)
    driver.get(TOURNAMENT_URL)
except:
    print("❌ Page load timeout! The site took too long to respond.")
    driver.quit()

# Step 3: Wait for JavaScript to Load & Find Leaderboard
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[starts-with(@id, 'list-item-') and contains(@class, 'list-row')]")))
    print("✅ Tournament leaderboard loaded successfully!")
except:
    print("❌ Data not found! The page might be blocking Selenium or loading too slowly.")
    driver.quit()

# Step 4: Extract Player Data (Live Output)
data = []

# **Find only actual player rows**
players = driver.find_elements(By.XPATH, "//div[starts-with(@id, 'list-item-') and contains(@class, 'list-row')]")
print(f"✅ Found {len(players)} players!")

for idx, player in enumerate(players):
    print(f"🔄 Processing Player {idx + 1}/{len(players)}...")

    # **Extract the player's unique ID from the `id` attribute**
    player_id = player.get_attribute("id").replace("list-item-", "").strip()
    print(f"   🔎 Extracting data for Player ID: {player_id}")

    # **SCROLL TO PLAYER (Ensures visibility before extracting data)**
    driver.execute_script("arguments[0].scrollIntoView();", player)
    time.sleep(1)  # Give time for elements to load

    # **Extract Position**
    try:
        pos = player.find_element(By.XPATH, f".//div[@id='list-item-{player_id}-position']").text.strip()
    except:
        pos = "N/A"

    # **Extract Player Name (From Nested <a> Tag's `title` Attribute)**
    try:
        name = player.find_element(By.XPATH, f".//div[@id='list-item-{player_id}-playername']//a").get_attribute("title").strip()
    except:
        name = "N/A"

    print(f"   ▶ Player: {name} (Position: {pos})")

    # **Extract Hole 1 - Hole 18 Scores**
    scores = []
    for hole in range(1, 19):
        try:
            hole_score = player.find_element(By.XPATH, f".//div[@id='list-item-{player_id}-h{hole}']").text.strip()
        except:
            hole_score = "N/A"
        scores.append(hole_score)

        # Print progress for each hole
        print(f"      🏌️‍♂️ Hole {hole}: {hole_score}")

    # **Extract Total Score (Tot)**
    try:
        total_score = player.find_element(By.XPATH, f".//div[@id='list-item-{player_id}-hTotal']").text.strip()
    except:
        total_score = "N/A"

    # **Extract Total to Par (TTP)**
    try:
        ttp = player.find_element(By.XPATH, f".//div[@id='list-item-{player_id}-totalToPar']").text.strip()
    except:
        ttp = "N/A"

    print(f"   🏆 Total: {total_score} | TTP: {ttp}")

    # **Append extracted data to list**
    row = [pos, name] + scores + [total_score, ttp]
    data.append(row)

# Step 5: Close the WebDriver
driver.quit()

# Step 6: Convert Data to DataFrame
columns = ["Pos", "Name"] + [f"H{i}" for i in range(1, 19)] + ["Tot", "TTP"]
df = pd.DataFrame(data, columns=columns)

# Step 7: Save CSV File
csv_filename = "tournament_results.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

# Step 8: Auto-download CSV in Google Colab
files.download(csv_filename)

print(f"✅ Data scraped successfully and downloaded: {csv_filename}!")




✅ Tournament leaderboard loaded successfully!
✅ Found 68 players!
🔄 Processing Player 1/68...
   🔎 Extracting data for Player ID: 38679262
   ▶ Player: GUNDERSEN, Marius Jørnsson (Position: 1)
      🏌️‍♂️ Hole 1: 3
      🏌️‍♂️ Hole 2: 4
      🏌️‍♂️ Hole 3: 2
      🏌️‍♂️ Hole 4: 3
      🏌️‍♂️ Hole 5: 6
      🏌️‍♂️ Hole 6: 3
      🏌️‍♂️ Hole 7: 3
      🏌️‍♂️ Hole 8: 4
      🏌️‍♂️ Hole 9: 4
      🏌️‍♂️ Hole 10: 2
      🏌️‍♂️ Hole 11: 4
      🏌️‍♂️ Hole 12: 4
      🏌️‍♂️ Hole 13: 3
      🏌️‍♂️ Hole 14: 4
      🏌️‍♂️ Hole 15: 4
      🏌️‍♂️ Hole 16: 4
      🏌️‍♂️ Hole 17: 5
      🏌️‍♂️ Hole 18: 4
   🏆 Total: 66 | TTP: -11
🔄 Processing Player 2/68...
   🔎 Extracting data for Player ID: 39223333
   ▶ Player: RASMUSSEN, Frithjof Åstorp (Position: T2)
      🏌️‍♂️ Hole 1: 5
      🏌️‍♂️ Hole 2: 3
      🏌️‍♂️ Hole 3: 3
      🏌️‍♂️ Hole 4: 4
      🏌️‍♂️ Hole 5: 3
      🏌️‍♂️ Hole 6: 3
      🏌️‍♂️ Hole 7: 3
      🏌️‍♂️ Hole 8: 4
      🏌️‍♂️ Hole 9: 5
      🏌️‍♂️ Hole 10: 3
      🏌️‍♂️ Hole 11: 4
    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Data scraped successfully and downloaded: tournament_results.csv!
